In [1]:
import gensim
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess
from smart_open import smart_open
import os

import warnings;
warnings.filterwarnings('ignore')

In [2]:
# List with 2 sentences
my_docs = ["Who let the dogs out?",
           "Who? Who? Who? Who?"]

# Tokenize the docs
tokenized_list = [simple_preprocess(doc) for doc in my_docs]

# Create the Corpus
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]
pprint(mycorpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(4, 4)]]


## Load textbook:

In [3]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    caching = True
    pagenos=set()

    for PageNumer,page in enumerate(PDFPage.get_pages(fp, pagenos , password=password,caching=caching, check_extractable=True)):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [4]:
from os import listdir
from os.path import isfile, join

DATA_DIR = "C:/Users/jry5/OneDrive/Desktop/1/"

# Grab the files
books = [pdf for pdf in listdir(DATA_DIR) if isfile(join(DATA_DIR, pdf))]
text = []

for book in books:
    path = DATA_DIR + book
    content = convert_pdf_to_txt(path)
    text.append(content)
    print ("\n----- Contents of %s -----\n" % book)
    print (content[:100])


----- Contents of 2017_Book_LinearRegression.pdf -----

David J. Olive

Linear 
Regression

Linear Regression

David J. Olive

Linear Regression

123

Da


In [5]:
# text = ['\n\n'.join(text)] 

## Find each paragraph:

In [7]:
import re
pat = re.compile(r'(?s)((?:[^\n][\n]?)+)', re.M)
text1 = pat.findall(text[0])

## Clean text:

In [8]:
import nltk
from nltk.corpus import stopwords
# stop_words = stopwords.words('english')

def clean_text(
    string: str, 
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:
    """
    A method to clean text 
    """
    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)

    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
            
    # remove backslash-apostrophe 
    string = re.sub("\'", "", string) 
    
    # remove everything except alphabets 
    string = re.sub("[^a-zA-Z]"," ",string) 
    
    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string        

In [9]:
data = []
count = 0
for i in range(len(text1)):
    data.append(clean_text(text1[i]).split())
for i in range(len(data)):
    count += data[i].count('bias')
count

2

## Create Bag of words:
## Build word2vec Embeddings

In [17]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api
model = Word2Vec(data, min_count = 10, workers=cpu_count())
model.save('model')

In [11]:
model['regression']

array([ 0.47447598, -0.50714296,  0.5753656 , -0.03402466,  0.44752416,
        0.72413677, -0.33330336,  0.51516443,  1.0497626 ,  0.8192747 ,
        0.09662783,  0.39593318, -0.47896796,  0.33325276,  0.94673544,
        0.19670576, -0.9646793 , -0.06000009,  0.19711249,  0.48845363,
        0.00256744, -0.37924588, -0.10640869,  0.3473871 ,  0.9649055 ,
       -0.6942536 ,  0.24901143, -0.76838386, -0.28010854, -0.30402097,
       -0.79135585,  0.8440176 ,  0.04658692,  0.28777543, -0.7958924 ,
       -0.12693746, -0.00620345, -0.0391525 ,  0.07302649,  0.5535461 ,
        0.05857614, -1.0168226 ,  0.13907485,  0.27156174,  0.3206015 ,
       -0.8867621 , -0.24572295, -0.7150718 ,  0.19871452, -0.10811483,
       -0.01955325, -0.3552129 , -0.7601704 ,  0.6217304 , -0.7315517 ,
       -0.6078159 ,  0.70880824, -0.38100582, -0.1633411 ,  0.58503085,
       -0.82559514, -0.11237887, -0.24963054, -0.01607132, -0.00995457,
        0.38562232,  0.83340436, -0.5828035 , -0.23919709, -1.26

In [12]:
model.most_similar('models')

[('multiple', 0.9537561535835266),
 ('multivariate', 0.9035055637359619),
 ('regression', 0.9006812572479248),
 ('special', 0.897425651550293),
 ('linear', 0.8899581432342529),
 ('generalized', 0.8808688521385193),
 ('analysis', 0.875869870185852),
 ('additive', 0.8687137365341187),
 ('alternatives', 0.8665547370910645),
 ('for', 0.8631240129470825)]

In [13]:
model.most_similar('regression')

[('linear', 0.9908047318458557),
 ('multiple', 0.9712517261505127),
 ('multivariate', 0.9667948484420776),
 ('models', 0.9006812572479248),
 ('poisson', 0.8598709106445312),
 ('doi', 0.8507057428359985),
 ('generalized', 0.8192943334579468),
 ('logistic', 0.8156444430351257),
 ('binary', 0.8149569034576416),
 ('simple', 0.8042111396789551)]

In [14]:
model.doesnt_match(['logistic', 'sample', 'simple', 'multiple'])

'sample'

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
print(float(cosine_similarity(model[['models']], model[['multiple']])))

0.9537561535835266


In [16]:
model.most_similar('models')

[('multiple', 0.9537561535835266),
 ('multivariate', 0.9035055637359619),
 ('regression', 0.9006812572479248),
 ('special', 0.897425651550293),
 ('linear', 0.8899581432342529),
 ('generalized', 0.8808688521385193),
 ('analysis', 0.875869870185852),
 ('additive', 0.8687137365341187),
 ('alternatives', 0.8665547370910645),
 ('for', 0.8631240129470825)]